## 맵핑 데이터 만들기
모델 평가에 활용하기 위해 mapping.json 파일을 제작
해당 데이터셋 폴더의 모든 파일들을 보면서 해당 파일의 경로 및 transcription을 맵핑할 예정
{data_name}_mapping.json << 이런 형태로 data/{data_name}_mapping.json 형태로 저장

### Import Library / package

In [1]:
# pip

In [2]:
# import
import os
import json

### Make File Path
특정 디렉토리 안에 있는 모든 flac 파일과 txt 파일의 경로를 배열로 만듦

##### **params**
* source_dir : 탐색할 디렉토리 경로

##### **returns**
* flac_path_list : source_dir 내의 모든 .flac 파일의 경로 배열
* trans_path_list : source_dir 내의 모든 .txt 파일의 경로 배열

In [3]:
def make_file_paths(source_dir):
    flac_path_list = []
    trans_path_list=[]
    for root, _, files in os.walk(source_dir):
            for file in files:
                if file.endswith(".flac"):
                    flac_source_path = os.path.join(root, file)
                    flac_path_list.append(flac_source_path)
                if file.endswith(".txt"):
                    trans_source_path = os.path.join(root, file)
                    trans_path_list.append(trans_source_path)
    return flac_path_list,trans_path_list

### Make Transcription Mapping Dict
make_file_paths를 활용해 만들어진 trans_path_list 내부의 경로들의 파일들을 한 줄씩 읽으면서 딕녀너리 반환

{파일이름 : 전사데이터} 

##### **Params**
* trans_path_list : 특정 디렉토리 내의 모든 전사 파일 경로

##### **returns**
* mapping_dict : {파일이름:전사데이터} 형태의 딕셔너리, 특정 디렉토리 내의 모든 txt에 대한 전사데이터를 딕셔너리 형태로 보유

In [4]:
def make_path_transcription_mapping(trans_path_list):
    mapping_dict = {}
    
    for trans in trans_path_list:
        with open(trans, "r", encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split(" ")
                file_name = parts[0]
                transcription = " ".join(parts[1:])
                mapping_dict[file_name] = transcription
                
    return mapping_dict

### Make Mapping Json File
make_path_transcription을 통해 만들어진 전사데이터와 make_file_paths을 통해 만들어진 flac_file_list를 활용하여 [{path:(flac_file_path),"transcription":(전사데이터)}] 형태의 배열을 json으로 변환해줌

##### **params**
* target_dir : mapping json 파일을 저장할 디렉토리 경로
* json_file_name :  {json_file_name}.json으로 저장할 때 사용됨
* flac_path_list : 맵핑될 음성 데이터 배열
* mapping_dict : make_path_transcription_mapping로 만들어진 {파일이름:전사데이터} 형태의 데이터

##### **returns**
* json 파일

In [5]:
def make_mapping_json(source_dir,json_file_name,flac_path_list,mapping_dict):
    mapped_list = []
    for flac_file in flac_path_list:
        file_name = os.path.splitext(os.path.basename(flac_file))[0]
        if file_name in mapping_dict:
            mapped_list.append({
                "path": flac_file,
                "transcription": mapping_dict[file_name]
            })

    if not os.path.exists(source_dir):
        os.makedirs(source_dir)
        
    output_json_path = os.path.join(source_dir, json_file_name+".json")
    
    with open(output_json_path, "w", encoding="utf-8") as json_file:
        json.dump(mapped_list, json_file, ensure_ascii=False, indent=4)
    
    print("Mapping saved to", output_json_path,"/",json_file_name)

In [6]:
### Make Sample Mapping Json File
# test_source_dir = "./data/test_data"
# test_json_source_dir = "./data/exist_test"
# test_json_file_name="rami_mapping"

# test_flac_path_list,test_trans_path_list = make_file_paths(test_source_dir) # set file path
# test_mapping_dict = make_path_transcription_mapping(test_trans_path_list)

# print(test_flac_path_list)
# print(test_trans_path_list)
# print(test_mapping_dict)
# make_mapping_json(test_json_source_dir,test_json_file_name,test_flac_path_list,test_mapping_dict)


In [7]:
source_dir_list = ["./data/x_eval","./data/x_eval-gaussian-10","./data/x_eval-gaussian-20","./data/x_eval-gaussian-30","./data/x_eval-gaussian-40","./data/x_eval-gaussian-50","./data/x_eval-gaussian-60","./data/x_eval-gaussian-70","./data/x_eval-gaussian-80","./data/x_eval-gaussian-90","./data/x_eval-gaussian-100","./data/x_eval-laplacian-10","./data/x_eval-laplacian-20","./data/x_eval-laplacian-30","./data/x_eval-laplacian-40","./data/x_eval-laplacian-50","./data/x_eval-laplacian-60","./data/x_eval-laplacian-70","./data/x_eval-laplacian-80","./data/x_eval-laplacian-90","./data/x_eval-laplacian-100"]
json_source_dir = "./data/mapping_jsons"
json_file_name_list=["eval","gaussian-10","gaussian-20","gaussian-30","gaussian-40","gaussian-50","gaussian-60","gaussian-70","gaussian-80","gaussian-90","gaussian-100","laplacian-10","laplacian-20","laplacian-30","laplacian-40","laplacian-50","laplacian-60","laplacian-70","laplacian-80","laplacian-90","laplacian-100",]

for idx,source_dir in enumerate(source_dir_list):
    flac_path_list,trans_path_list = make_file_paths(source_dir)
    mapping_dict = make_path_transcription_mapping(trans_path_list)
    make_mapping_json(json_source_dir,json_file_name_list[idx],flac_path_list,mapping_dict)

Mapping saved to ./data/mapping_jsons/eval.json / eval
Mapping saved to ./data/mapping_jsons/gaussian-10.json / gaussian-10
Mapping saved to ./data/mapping_jsons/gaussian-20.json / gaussian-20
Mapping saved to ./data/mapping_jsons/gaussian-30.json / gaussian-30
Mapping saved to ./data/mapping_jsons/gaussian-40.json / gaussian-40
Mapping saved to ./data/mapping_jsons/gaussian-50.json / gaussian-50
Mapping saved to ./data/mapping_jsons/gaussian-60.json / gaussian-60
Mapping saved to ./data/mapping_jsons/gaussian-70.json / gaussian-70
Mapping saved to ./data/mapping_jsons/gaussian-80.json / gaussian-80
Mapping saved to ./data/mapping_jsons/gaussian-90.json / gaussian-90
Mapping saved to ./data/mapping_jsons/gaussian-100.json / gaussian-100
Mapping saved to ./data/mapping_jsons/laplacian-10.json / laplacian-10
Mapping saved to ./data/mapping_jsons/laplacian-20.json / laplacian-20
Mapping saved to ./data/mapping_jsons/laplacian-30.json / laplacian-30
Mapping saved to ./data/mapping_jsons/la